In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings, string
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import string, nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import nltk
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
nltk.download('omw-1.4')


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/pranjalmishra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/pranjalmishra/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
df = pd.read_csv(r'/Users/pranjalmishra/Downloads/prj3/Preprocessed Fake Reviews Detection Dataset.csv')
df

,Unnamed: 0,category,rating,label,text_
0,0,Home_and_Kitchen_5,5.0,CG,love well made sturdy comfortable i love very ...
1,1,Home_and_Kitchen_5,5.0,CG,love great upgrade original i 've mine couple ...
2,2,Home_and_Kitchen_5,5.0,CG,this pillow saved back i love look feel pillow
3,3,Home_and_Kitchen_5,1.0,CG,missing information use great product price i
4,4,Home_and_Kitchen_5,5.0,CG,very nice set good quality we set two month
...,...,...,...,...,...
40427,40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,i read review saying bra ran small i ordered t...
40428,40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,i n't sure exactly would it little large small...
40429,40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,you wear hood wear hood wear jacket without ho...
40430,40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,i liked nothing dress the reason i gave star i...


In [5]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,love well made sturdy comfortable i love very ...
1,Home_and_Kitchen_5,5.0,CG,love great upgrade original i 've mine couple ...
2,Home_and_Kitchen_5,5.0,CG,this pillow saved back i love look feel pillow
3,Home_and_Kitchen_5,1.0,CG,missing information use great product price i
4,Home_and_Kitchen_5,5.0,CG,very nice set good quality we set two month


In [6]:
df.dropna(inplace=True)
df['length'] = df['text_'].apply(len)

In [7]:
df[df['label']=='OR'][['text_','length']].sort_values(by='length',ascending=False).head().iloc[0].text_

"weak on current science after seeing twice i agree much positive five star review out respect read review i 'll repeat everything i like presentation i found goofy oversize earring hairdo facial hair arrangement daniel vitalis described `` wild food expert '' distracting ugh ditto david wolfe extremely goofy wild hairdo on hand jon gabriel described `` author weight loss expert '' nicely groomed good presenter his story personal transformation fellow pound whew becoming jock normal weight inspiring christiane northrup preserve rank one america 's cutest doctor a really nice looking woman presentation dr. mercola jason vale kris carr alejandro junger fine it disappointing jamie oliver popular uk give baby cow growth fluid pas unscientific popular idea milk none presenter anything zilch say work doctor t. colin campbell milk body bad it good see presenter take stand sugar they agreed evil sugar refined carbohydrate with respect dr. northrup `` it 's fat make fat 's sugar '' statement pa

In [8]:
def text_process(review):
    nopunc = [char for char in review if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [9]:
  bow_transformer = CountVectorizer(analyzer=text_process)
  bow_transformer

CountVectorizer(analyzer=<function text_process at 0x3058c74c0>)

In [10]:
bow_transformer.fit(df['text_'])
print("Total Vocabulary:",len(bow_transformer.vocabulary_))

Total Vocabulary: 42783


In [11]:
review4 = df['text_'][3]
review4

'missing information use great product price i'

In [12]:
bow_msg4 = bow_transformer.transform([review4])
print(bow_msg4)
print(bow_msg4.shape)

  (0, 16958)	1
  (0, 19702)	1
  (0, 24580)	1
  (0, 29428)	1
  (0, 29586)	1
  (0, 40168)	1
(1, 42783)


In [13]:
index_to_word = {index: word for word, index in bow_transformer.vocabulary_.items()}
print(index_to_word[15841])
print(index_to_word[23848])

ful
meanspiritedness


In [14]:
bow_reviews = bow_transformer.transform(df['text_'])

In [15]:
print("Shape of Bag of Words Transformer for the entire reviews corpus:",bow_reviews.shape)
print("Amount of non zero values in the bag of words model:",bow_reviews.nnz)

Shape of Bag of Words Transformer for the entire reviews corpus: (40431, 42783)
Amount of non zero values in the bag of words model: 1020945


In [16]:
print("Sparsity:",np.round((bow_reviews.nnz/(bow_reviews.shape[0]*bow_reviews.shape[1]))*100,2))

Sparsity: 0.06


In [17]:
tfidf_transformer = TfidfTransformer().fit(bow_reviews)
tfidf_rev4 = tfidf_transformer.transform(bow_msg4)
print(bow_msg4)

  (0, 16958)	1
  (0, 19702)	1
  (0, 24580)	1
  (0, 29428)	1
  (0, 29586)	1
  (0, 40168)	1


In [18]:
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['mango']])
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['book']])

10.91422964906803
2.821837302066699


In [19]:
tfidf_reviews = tfidf_transformer.transform(bow_reviews)
print("Shape:",tfidf_reviews.shape)
print("No. of Dimensions:",tfidf_reviews.ndim)

Shape: (40431, 42783)
No. of Dimensions: 2


In [20]:
review_train, review_test, label_train, label_test = train_test_split(df['text_'],df['label'],test_size=0.35)

In [21]:
# SVM Pipeline setup
svm_pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # Converts text to a matrix of token counts
    ('tfidf', TfidfTransformer()),                    # Apply TF-IDF
    ('classifier', SVC())                             # Apply Support Vector Machine
])

# Training the SVM pipeline
svm_pipeline.fit(review_train, label_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x3058c74c0>)),
                ('tfidf', TfidfTransformer()), ('classifier', SVC())])

In [22]:
# Function to classify a user-input review using the trained SVM pipeline
def classify_review():
    """
    Prompts the user to enter a review and classifies it as 'Original' or 'Computer-generated'.
    """
    new_review = input("Enter your review: ")  # User inputs the review
    prediction = svm_pipeline.predict([new_review])[0]  # Prediction using the SVM model
    return f"The review is predicted as: {'Original' if prediction == 'OR' else 'Computer-generated'}"

# Running the function to classify user-input review
if __name__ == "__main__":
    print(classify_review())

The review is predicted as: Computer-generated


In [23]:
from joblib import dump, load
# Save the model to disk
dump(svm_pipeline, 'svm_model.joblib')


['svm_model.joblib']